In [4]:
%load_ext autoreload
import cobra
import numpy as np
from importlib import reload
import mcs
from scipy import sparse
#import traceback
#import warnings
import sys

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


This is a bilevel-example with 1 Target and 1 Desired region

In [6]:
%autoreload
# load network
network = cobra.io.read_sbml_model("weak_coupling.sbml")
# mcs.fba(network,constr=["r1=0"])
# specify modules
# specify modules
modules = [mcs.SD_Module(network,"mcs_bilvl",module_sense="target",constraints=["r_P <= 0.5"],inner_objective="-r_BM")]
modules.append(mcs.SD_Module(network,"mcs_lin",module_sense="desired",constraints="r_BM >= 1"))

# specify MCS setup
ko_cost = {'r2': 1, 'r3': 1, 'r4': 1, 'r5': 1, 'r6': 1, 'r7': 1, 'r8': 1, 'r9': 1}
ki_cost = {'r1': 1}
maxSolutions = 50
maxCost = 50
solver = 'cplex'

# construct MCS MILP
mcsEnum = mcs.StrainDesigner(network,modules,
       ko_cost=ko_cost,ki_cost=ki_cost,max_cost=maxCost,M=None,
       solver=solver)

# solve MILP
# mcsEnum.enumerate()
# mcsEnum.compute()
mcsEnum.compute_optimal()

TypeError: 

This is a bilevel-example with only 1 desired region

In [3]:
%autoreload
# load network
network = cobra.io.read_sbml_model("weak_coupling.sbml")
# mcs.fba(network,constr=["r1=0"])
# specify modules
modules = mcs.SD_Module(network,"mcs_bilvl",module_sense="desired",constraints=["r_BM >= 0.5","r_P >= 1"],inner_objective="-r_BM")

# specify MCS setup
ko_cost = {'r2': 1, 'r3': 1, 'r4': 1, 'r5': 1, 'r6': 1, 'r7': 1, 'r8': 1, 'r9': 1}
ki_cost = {'r1': 1}
maxSolutions = 50
maxCost = 50
solver = 'scip'

# construct MCS MILP
mcsEnum = mcs.StrainDesignMILP(network,modules,ko_cost=ko_cost,ki_cost=ki_cost,max_cost=7,solver=solver,M=None)

# solve MILP
mcsEnum.compute_optimal()

Constructing MILP.
Bounding MILP.
Finding optimal strain designs ...
Strain design with cost 1: {'r3': -1}
Strain design with cost 1: {'r4': -1}
Finished. 2 solutions found.


([{'r3': -1.0}, {'r4': -1.0}], 0)

This is a regular MCS example with 1 target and 1 desired region (HERE SCIP STILL SEEMS TO TAG VALID MCS AS INVALID)

In [24]:
%autoreload
# load network
network = cobra.io.read_sbml_model("SmallExample.sbml")

# specify modules
modules = [mcs.SD_Module(network,"mcs_lin",module_sense="target",constraints=["R04 >= 1","2 R01 >= 0.5"])]
modules.append(mcs.SD_Module(network,"mcs_lin",module_sense="desired",constraints="R03 >= 1"))

# specify MCS setup
ko_cost = {'R05': 2, 'R06': 3, 'R07': 4, 'R08': 4, 'R09': 4, 'R10': 4}
ki_cost = {'R02': 6}
maxSolutions = 8
maxCost = 8
solver = 'scip'

# construct MCS MILP
mcsEnum = mcs.StrainDesignMILP(network,modules,ko_cost=ko_cost,ki_cost=ki_cost,max_cost=maxCost,solver=solver,M=None)

# solve MILP
# mcsEnum.enumerate()
# mcsEnum.compute()
mcsEnum.compute_optimal()

Constructing MILP.
Bounding MILP.
Finding optimal strain designs ...
Strain design with cost 4: {'R07': -1}
Strain design with cost 4: {'R10': -1}
Strain design with cost 7: {'R06': -1, 'R08': -1}
Finished. 3 solutions found.


([{'R07': -1.0}, {'R10': -1.0}, {'R06': -1.0, 'R08': -1.0}], 0)

This is a bilevel-example with 1 Target and 1 Desired region

In [12]:
%autoreload
# load network
network = cobra.io.read_sbml_model("weak_coupling.sbml")
mcs.fba(network,constr=["r1=0"])
# specify modules
# specify modules
modules = mcs.SD_Module(network,"mcs_bilvl",module_sense="target",constraints=["r_P <= 0.5"],inner_objective="-r_BM")

# specify MCS setup
maxSolutions = 6
maxCost = 3
solver = 'cplex'

# construct MCS MILP
mcsEnum = mcs.StrainDesignMILP(network,modules,max_cost=maxCost,M=None,solver=solver)

# solve MILP
# mcsEnum.compute()
mcsEnum.compute(max_solutions=maxSolutions)
# mcsEnum.enumerate(max_solutions=maxSolutions)
# mcsEnum.enumerate(max_solutions=maxSolutions)
# mcsEnum.compute_optimal(max_solutions=maxSolutions)

Constructing MILP.
Bounding MILP.
Finding (also non-optimal) strain designs ...
Searching in full search space.
Minimizing number of interventions in subspace with 3 possible targets.
Strain design with cost 1: {'r2': -1}
Searching in full search space.
Minimizing number of interventions in subspace with 1 possible targets.
Strain design with cost 1: {'r_S': -1}
Searching in full search space.
Minimizing number of interventions in subspace with 3 possible targets.
Strain design with cost 3: {'r4': -1, 'r6': -1, 'r_Q': -1}
Searching in full search space.
Minimizing number of interventions in subspace with 3 possible targets.
Strain design with cost 2: {'r_BM': -1, 'r_Q': -1}
Searching in full search space.
Minimizing number of interventions in subspace with 3 possible targets.
Strain design with cost 3: {'r4': -1, 'r7': -1, 'r_Q': -1}
Searching in full search space.
Minimizing number of interventions in subspace with 2 possible targets.
Strain design with cost 2: {'r4': -1, 'r_BM': -1}


([{'r2': -1.0},
  {'r_S': -1.0},
  {'r4': -1.0, 'r6': -1.0, 'r_Q': -1.0},
  {'r_BM': -1.0, 'r_Q': -1.0},
  {'r4': -1.0, 'r7': -1.0, 'r_Q': -1.0},
  {'r4': -1.0, 'r_BM': -1.0}],
 0)

This is a mid-scale-example with only 1 desired region

In [6]:
%autoreload
# load network
network = cobra.io.read_sbml_model("ECC2comp.sbml")
# remove external metabolites and obsolete reactions
external_mets = [i for i,cpts in zip(network.metabolites,network.metabolites.list_attr("compartment")) if cpts == 'External_Species']
network.remove_metabolites(external_mets)
S = cobra.util.create_stoichiometric_matrix(network)
obsolete_reacs = [reac for reac,b_rempty in zip(network.reactions,np.any(S,0)) if not b_rempty]
network.remove_reactions(obsolete_reacs)
# close some uptakes
network.reactions.AcUp.upper_bound = 0
network.reactions.GlycUp.upper_bound = 0

sol = mcs.fba(network,constr=["O2Up=0"])
# specify modules
modules = mcs.SD_Module(network,"mcs_bilvl",module_sense="desired",constraints=["2 Growth >= 0.1","EthEx >= 1"],inner_objective="Growth")

network.remove_metabolites
network.metabolites.list_attr("compartment")
# specify MCS setup
maxSolutions = 20
maxCost = 10
solver = 'gurobi'

# construct MCS MILP
mcsEnum = mcs.StrainDesigner(network,modules,max_cost=maxCost,solver=solver,M=None)

# solve MILP
# mcsEnum.enumerkate(max_solutions=maxSolutions)
# mcsEnum.compute_optimal(max_solutions=maxSolutions,time_limit=15)
mcsEnum.compute(max_solutions=maxSolutions,time_limit=15)

Read LP format model from file C:\Users\phili\AppData\Local\Temp\tmp2pd9du59.lp
Reading time = 0.02 seconds
: 54 rows, 164 columns, 562 nonzeros
Constructing MILP.
Bounding MILP.
Finding (also non-optimal) strain designs ...
Searching in full search space.
Minimizing number of interventions in subspace with 10 possible targets.
Strain design with cost 5: {'HEX1': -1, 'MALS': -1, 'PGI': -1, 'PPS': -1, 'TKT1': -1}
Strain design with cost 6: {'AKGDH': -1, 'HEX1': -1, 'MALS': -1, 'PPS': -1, 'THD2pp': -1, 'TKT1': -1}
Searching in full search space.
Minimizing number of interventions in subspace with 10 possible targets.
Strain design with cost 4: {'FBA': -1, 'GLCt2pp': -1, 'MALS': -1, 'PPS': -1}
Searching in full search space.
Minimizing number of interventions in subspace with 10 possible targets.
Strain design with cost 4: {'h_pEx': -1, 'MALS': -1, 'SUCOAS': -1, 'TKT1': -1}
Strain design with cost 5: {'FBA': -1, 'ME2': -1, 'SUCOAS': -1, 'THD2pp': -1, 'TKT1': -1}
Strain design with cost 5:

([{'HEX1': -1.0, 'MALS': -1.0, 'PGI': -1.0, 'PPS': -1.0, 'TKT1': -1.0},
  {'AKGDH': -1.0,
   'HEX1': -1.0,
   'MALS': -1.0,
   'PPS': -1.0,
   'THD2pp': -1.0,
   'TKT1': -1.0},
  {'FBA': -1.0, 'GLCt2pp': -1.0, 'MALS': -1.0, 'PPS': -1.0},
  {'h_pEx': -1.0, 'MALS': -1.0, 'SUCOAS': -1.0, 'TKT1': -1.0},
  {'FBA': -1.0, 'ME2': -1.0, 'SUCOAS': -1.0, 'THD2pp': -1.0, 'TKT1': -1.0},
  {'ME2': -1.0, 'PGM': -1.0, 'SUCOAS': -1.0, 'THD2pp': -1.0, 'TKT1': -1.0},
  {'AcEx': -1.0, 'ICL': -1.0, 'SUCOAS': -1.0, 'TKT1': -1.0},
  {'h2oEx': -1.0},
  {'HEX1': -1.0, 'MDH': -1.0, 'PPS': -1.0},
  {'h_pEx': -1.0, 'ICL': -1.0, 'SUCOAS': -1.0, 'TKT1': -1.0},
  {'NADH16pp': -1.0, 'NADH17pp': -1.0, 'THD2pp': -1.0, 'TKT1': -1.0},
  {'NADH16pp': -1.0, 'NADH17pp': -1.0, 'PGI': -1.0, 'THD2pp': -1.0},
  {'FRD2': -1.0, 'NADH16pp': -1.0, 'THD2pp': -1.0, 'TKT1': -1.0},
  {'FRD2': -1.0, 'NADH16pp': -1.0, 'PGI': -1.0, 'THD2pp': -1.0},
  {'h_pEx': -1.0,
   'NADH16pp': -1.0,
   'NADH17pp': -1.0,
   'PPC': -1.0,
   'THD2pp': -1

In [ ]:
%autoreload
# load network
network = cobra.io.read_sbml_model("ECC2comp.sbml")
# remove external metabolites and obsolete reactions
external_mets = [i for i,cpts in zip(network.metabolites,network.metabolites.list_attr("compartment")) if cpts == 'External_Species']
network.remove_metabolites(external_mets)
S = cobra.util.create_stoichiometric_matrix(network)
obsolete_reacs = [reac for reac,b_rempty in zip(network.reactions,np.any(S,0)) if not b_rempty]
network.remove_reactions(obsolete_reacs)
# close some uptakes
network.reactions.AcUp.upper_bound = 0
network.reactions.GlycUp.upper_bound = 0
network.reactions.ATPM.upper_bound = 1000
network.reactions.ATPM.lower_bound = 3.15

sol = mcs.fba(network,constr=["O2Up=0"])
# specify modules
modules = mcs.SD_Module(network,"mcs_lin",module_sense="target")

network.remove_metabolites
network.metabolites.list_attr("compartment")
# specify MCS setup
maxSolutions = 4
maxCost = 50
solver = 'cplex'

# construct MCS MILP
mcsEnum = mcs.StrainDesignMILP(network,modules,max_cost=7,solver=solver,M=None)

# solve MILP
# mcsEnum.enumerate(max_solutions=maxSolutions)
# mcsEnum.compute_optimal(max_solutions=maxSolutions)
mcsEnum.compute(max_solutions=maxSolutions)

Constructing MILP.
Bounding MILP.
Finding (also non-optimal) strain designs ...
Searching in full search space.
Minimizing number of interventions in subspace with 1 possible targets.
Strain design with cost 1: {'ATPM': -1}
Searching in full search space.
Minimizing number of interventions in subspace with 6 possible targets.
Strain design with cost 4: {'ACKr': -1, 'AKGDH': -1, 'ATPS4rpp': -1, 'ENO': -1}
Searching in full search space.
Minimizing number of interventions in subspace with 4 possible targets.
Strain design with cost 4: {'AKGDH': -1, 'ATPS4rpp': -1, 'ENO': -1, 'PTAr': -1}
Searching in full search space.
Minimizing number of interventions in subspace with 5 possible targets.
Strain design with cost 4: {'ACKr': -1, 'ATPS4rpp': -1, 'ENO': -1, 'SUCOAS': -1}
Finished. 4 solutions found.


([{'ATPM': -1.0},
  {'ACKr': -1.0, 'AKGDH': -1.0, 'ATPS4rpp': -1.0, 'ENO': -1.0},
  {'AKGDH': -1.0, 'ATPS4rpp': -1.0, 'ENO': -1.0, 'PTAr': -1.0},
  {'ACKr': -1.0, 'ATPS4rpp': -1.0, 'ENO': -1.0, 'SUCOAS': -1.0}],
 0)

In [ ]:
%autoreload
# load network
network = cobra.io.read_sbml_model("ECC2comp.sbml")
# remove external metabolites and obsolete reactions
external_mets = [i for i,cpts in zip(network.metabolites,network.metabolites.list_attr("compartment")) if cpts == 'External_Species']
network.remove_metabolites(external_mets)
S = cobra.util.create_stoichiometric_matrix(network)
obsolete_reacs = [reac for reac,b_rempty in zip(network.reactions,np.any(S,0)) if not b_rempty]
network.remove_reactions(obsolete_reacs)
# close some uptakes
network.reactions.AcUp.upper_bound = 0
network.reactions.SuccUp.upper_bound = 0
network.reactions.GlycUp.upper_bound = 0
network.reactions.CO2Up = 0
network.reactions.ATPM.upper_bound = 1000
network.reactions.ATPM.lower_bound = 3.15

# sol = mcs.fba(network,constr=["O2Up=0"])
# sol1 = mcs.fba(network,constr=['AcEx = 0','ATPS4rpp = 0', 'G3PD5= 0', 'NADTRHD= 0', 'PFL= 0', 'PYK= 0', 'SuccEx= 0', 'SUCCt2_2pp= 0'])
# sol2 = mcs.fva(network,constr=['Growth = '+str(0.999*sol1.objective_value),'AcEx = 0','ATPS4rpp = 0', 'G3PD5= 0', 'NADTRHD= 0', 'PFL= 0', 'PYK= 0', 'SuccEx= 0', 'SUCCt2_2pp= 0'])

# specify modules
modules = mcs.SD_Module(network,"optknock",constraints=["Growth >= 0.2","EthEx >= 1"],inner_objective="Growth",outer_objective="EthEx")

# specify MCS setup
maxSolutions = 100
maxCost = 2
solver = 'scip'

# construct MCS MILP
mcsEnum = mcs.StrainDesignMILP(network,modules,max_cost=maxCost,M=None, solver=solver)

# solve MILP
# mcsEnum.compute(max_solutions=maxSolutions)
# mcsEnum.compute_optimal(max_solutions=maxSolutions)
mcsEnum.enumerate(max_solutions=maxSolutions)

Constructing MILP.
Bounding MILP.


c:\users\phili\documents\python\mcs\mcs\strainDesigner.py:332: UserWarning: SCIP does not natively support solution pool generation. An high-level implementation of populate is used. Consider using compute_optimal instead of enumerate, as it returns the same results but faster.
  "Consider using compute_optimal instead of enumerate, as " + \


Enumerating strain designs ...
Enumerating all solutions with the objective value -13.868049181833067
Strain designs with cost 2.0: [{'AcEx': -1.0, 'CYTBO3_4pp': -1.0}]
Strain designs with cost 2.0: [{'ACKr': -1.0, 'CYTBO3_4pp': -1.0}]
Strain designs with cost 2.0: [{'AcEx': -1.0, 'O2Up': -1.0}]
Enumerating all solutions with the objective value -13.77629593754908
Strain designs with cost 2.0: [{'ACKr': -1.0, 'O2Up': -1.0}]
Strain designs with cost 2.0: [{'CYTBO3_4pp': -1.0, 'PTAr': -1.0}]
Strain designs with cost 2.0: [{'O2Up': -1.0, 'PTAr': -1.0}]
Enumerating all solutions with the objective value -11.602860050576796
Strain designs with cost 2.0: [{'AcEx': -1.0, 'h2oEx': -1.0}]
Enumerating all solutions with the objective value -11.599739089700533
Strain designs with cost 2.0: [{'h2oEx': -1.0, 'THD2pp': -1.0}]
Enumerating all solutions with the objective value -11.520532158562109
Strain designs with cost 2.0: [{'ACKr': -1.0, 'h2oEx': -1.0}]
Strain designs with cost 2.0: [{'h2oEx': -1

([{'AcEx': -1.0, 'CYTBO3_4pp': -1.0},
  {'ACKr': -1.0, 'CYTBO3_4pp': -1.0},
  {'AcEx': -1.0, 'O2Up': -1.0},
  {'ACKr': -1.0, 'O2Up': -1.0},
  {'CYTBO3_4pp': -1.0, 'PTAr': -1.0},
  {'O2Up': -1.0, 'PTAr': -1.0},
  {'AcEx': -1.0, 'h2oEx': -1.0},
  {'h2oEx': -1.0, 'THD2pp': -1.0},
  {'ACKr': -1.0, 'h2oEx': -1.0},
  {'h2oEx': -1.0, 'PTAr': -1.0},
  {'ATPS4rpp': -1.0, 'h2oEx': -1.0},
  {'O2Up': -1.0, 'THD2pp': -1.0},
  {'CYTBO3_4pp': -1.0, 'THD2pp': -1.0},
  {'CYTBO3_4pp': -1.0, 'PPC': -1.0},
  {'O2Up': -1.0, 'PPC': -1.0},
  {'h2oEx': -1.0, 'TKT2': -1.0},
  {'h2oEx': -1.0},
  {'h2oEx': -1.0, 'PGL': -1.0},
  {'h2oEx': -1.0, 'SUCOAS': -1.0},
  {'h2oEx': -1.0, 'NADTRHD': -1.0},
  {'EDA': -1.0, 'h2oEx': -1.0},
  {'G6PDH2r': -1.0, 'h2oEx': -1.0},
  {'h2oEx': -1.0, 'RPE': -1.0},
  {'CYTBO3_4pp': -1.0, 'PYK': -1.0},
  {'O2Up': -1.0, 'PYK': -1.0},
  {'O2Up': -1.0, 'TKT2': -1.0},
  {'O2Up': -1.0, 'SUCOAS': -1.0},
  {'CYTBO3_4pp': -1.0, 'TKT2': -1.0},
  {'CYTBO3_4pp': -1.0, 'h2Ex': -1.0},
  {'CO2Ex': 

In [ ]:
%autoreload
# load network
network = cobra.io.read_sbml_model("ECC2comp.sbml")
# remove external metabolites and obsolete reactions
external_mets = [i for i,cpts in zip(network.metabolites,network.metabolites.list_attr("compartment")) if cpts == 'External_Species']
network.remove_metabolites(external_mets)
S = cobra.util.create_stoichiometric_matrix(network)
obsolete_reacs = [reac for reac,b_rempty in zip(network.reactions,np.any(S,0)) if not b_rempty]
network.remove_reactions(obsolete_reacs)
# close some uptakes
network.reactions.AcUp.upper_bound = 0
network.reactions.SuccUp.upper_bound = 0
network.reactions.GlycUp.upper_bound = 0
network.reactions.CO2Up = 0
network.reactions.ATPM.upper_bound = 1000
network.reactions.ATPM.lower_bound = 3.15
# for i in enumerate(network.reactions)

# sol = mcs.fba(network,constr=["O2Up=0"])
# sol1 = mcs.fba(network,constr=['AcEx = 0','ATPS4rpp = 0', 'G3PD5= 0', 'NADTRHD= 0', 'PFL= 0', 'PYK= 0', 'SuccEx= 0', 'SUCCt2_2pp= 0'])
# sol2 = mcs.fva(network,constr=['Growth = '+str(0.999*sol1.objective_value),'AcEx = 0','ATPS4rpp = 0', 'G3PD5= 0', 'NADTRHD= 0', 'PFL= 0', 'PYK= 0', 'SuccEx= 0', 'SUCCt2_2pp= 0'])

# specify modules
modules = mcs.SD_Module(network,"robustknock",constraints=["Growth >= 0.2","EthEx >= 1"],inner_objective="Growth",outer_objective="EthEx")

# specify MCS setup
maxSolutions = 10
maxCost = 15
solver = 'gurobi'

# construct MCS MILP
mcsEnum = mcs.StrainDesignMILP(network,modules,max_cost=maxCost,M=None, solver=solver)

# solve MILP
# mcsEnum.compute(max_solutions=maxSolutions)
mcsEnum.compute_optimal(max_solutions=maxSolutions)
# mcsEnum.enumerate(max_solutions=maxSolutions)

In [13]:
%autoreload
# load network
network = cobra.io.read_sbml_model("ECC2comp.sbml")
# remove external metabolites and obsolete reactions
external_mets = [i for i,cpts in zip(network.metabolites,network.metabolites.list_attr("compartment")) if cpts == 'External_Species']
network.remove_metabolites(external_mets)
S = cobra.util.create_stoichiometric_matrix(network)
obsolete_reacs = [reac for reac,b_rempty in zip(network.reactions,np.any(S,0)) if not b_rempty]
network.remove_reactions(obsolete_reacs)
# close some uptakes
network.reactions.AcUp.upper_bound = 0
network.reactions.SuccUp.upper_bound = 0
network.reactions.GlycUp.upper_bound = 0
network.reactions.CO2Up = 0
network.reactions.ATPM.upper_bound = 1000
network.reactions.ATPM.lower_bound = 3.15
# for i in enumerate(network.reactions)

# sol = mcs.fba(network,constr=["O2Up=0"])
# sol1 = mcs.fba(network,constr=['AcEx = 0','ATPS4rpp = 0', 'G3PD5= 0', 'NADTRHD= 0', 'PFL= 0', 'PYK= 0', 'SuccEx= 0', 'SUCCt2_2pp= 0'])
# sol2 = mcs.fva(network,constr=['Growth = '+str(0.999*sol1.objective_value),'AcEx = 0','ATPS4rpp = 0', 'G3PD5= 0', 'NADTRHD= 0', 'PFL= 0', 'PYK= 0', 'SuccEx= 0', 'SUCCt2_2pp= 0'])

# specify modules
modules = mcs.SD_Module(network,"optcouple",constraints=["Growth >= 0.2","EthEx >= 1"],inner_objective="Growth",prod_id="EthEx",min_gcp=0.4)

# specify MCS setup
maxSolutions = 10
maxCost = 15
solver = 'scip'

# construct MCS MILP
mcsEnum = mcs.StrainDesignMILP(network,modules,max_cost=maxCost,M=None, solver=solver)

# solve MILP
mcsEnum.compute(max_solutions=maxSolutions)
# mcsEnum.compute_optimal(max_solutions=maxSolutions)
# mcsEnum.enumerate(max_solutions=maxSolutions)

Constructing MILP.
Bounding MILP.
Finding (also non-optimal) strain designs ...
Searching in full search space.
Found solution with objective value -0.7232319930893569
Minimizing number of interventions in subspace with 12 possible targets.
Strain design with cost 7: {'ACKr': -1, 'FRD2': -1, 'GLCt2pp': -1, 'LacEx': -1, 'MDH': -1, 'POX': -1, 'PPS': -1}
Strain design with cost 7: {'ACKr': -1, 'GLCt2pp': -1, 'LacEx': -1, 'MDH': -1, 'NADH17pp': -1, 'POX': -1, 'PPS': -1}
Searching in full search space.
Found solution with objective value -0.47067448500915887
Minimizing number of interventions in subspace with 12 possible targets.
Strain design with cost 7: {'FUM': -1, 'HEX1': -1, 'LDH_D': -1, 'MALS': -1, 'POX': -1, 'PPS': -1, 'PTAr': -1}
Strain design with cost 7: {'ACKr': -1, 'FUM': -1, 'HEX1': -1, 'LDH_D': -1, 'MALS': -1, 'POX': -1, 'PPS': -1}
Searching in full search space.
Found solution with objective value -0.4940321270089865
Minimizing number of interventions in subspace with 15 poss

([{'ACKr': -1.0,
   'FRD2': -1.0,
   'GLCt2pp': -1.0,
   'LacEx': -1.0,
   'MDH': -1.0,
   'POX': -1.0,
   'PPS': -1.0},
  {'ACKr': -1.0,
   'GLCt2pp': -1.0,
   'LacEx': -1.0,
   'MDH': -1.0,
   'NADH17pp': -1.0,
   'POX': -1.0,
   'PPS': -1.0},
  {'FUM': -1.0,
   'HEX1': -1.0,
   'LDH_D': -1.0,
   'MALS': -1.0,
   'POX': -1.0,
   'PPS': -1.0,
   'PTAr': -1.0},
  {'ACKr': -1.0,
   'FUM': -1.0,
   'HEX1': -1.0,
   'LDH_D': -1.0,
   'MALS': -1.0,
   'POX': -1.0,
   'PPS': -1.0},
  {'AcEx': -1.0,
   'FRD2': -1.0,
   'LDH_D': -1.0,
   'MDH': -1.0,
   'PGI': -1.0,
   'PPS': -1.0,
   'TKT1': -1.0},
  {'AcEx': -1.0,
   'LDH_D': -1.0,
   'MDH': -1.0,
   'NADH17pp': -1.0,
   'PGI': -1.0,
   'PPS': -1.0,
   'TKT1': -1.0},
  {'GND': -1.0,
   'ICL': -1.0,
   'LDH_D': -1.0,
   'MDH': -1.0,
   'PGI': -1.0,
   'POX': -1.0,
   'PPS': -1.0,
   'PTAr': -1.0},
  {'AcEx': -1.0,
   'FRD2': -1.0,
   'GLCt2pp': -1.0,
   'LacEx': -1.0,
   'MDH': -1.0,
   'PPS': -1.0},
  {'AcEx': -1.0,
   'GLCt2pp': -1.0,
   '